<a href="https://colab.research.google.com/github/anil-chhetri/NLP/blob/master/spam_and_ham_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [138]:
import re
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [139]:
!cp "/content/gdrive/My Drive/Datasets/smsclassification.csv" /content/

In [140]:
df = pd.read_csv('smsclassification.csv', names=['result', 'features', 'unamed1', 'unamed2', 'unamed3'], skiprows=1)

In [141]:
df.head()

,result,features,unamed1,unamed2,unamed3
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [142]:
from sklearn.preprocessing import LabelEncoder
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [143]:
lr = LabelEncoder()
target = lr.fit_transform(df['result'])
lr

LabelEncoder()

In [144]:
lr.classes_

array(['ham', 'spam'], dtype=object)

In [145]:
stopwords.words

<bound method WordListCorpusReader.words of <WordListCorpusReader in '/root/nltk_data/corpora/stopwords'>>

In [146]:
features = df['features']

In [147]:
 re.sub(r"((?<=^)|(?<= )).((?=$)|(?= ))", ' ', 'U dun say so early hor... U u c already then say...')

'  dun say so early hor...       already then say...'

In [148]:
final_text = []

for index in range(0,len(features)):

  ## removing symbols and numbers
  clean_text = re.sub(r'\W|\d', ' ', features[index])

  ## removing single word
  #clean_text = re.sub(r'\s+[a-zA-Z]\s+', ' ', clean_text)
  clean_text = re.sub(r"((?<=^)|(?<= )).((?=$)|(?= ))", ' ', clean_text)

  ## removing multiple spaces
  clean_text = re.sub(r'\s+', ' ', clean_text)

  ## converting to lower
  clean_text = clean_text.lower()

  ## removing stop words
  #print(clean_text)
  clean_text = ' '.join([x for x in word_tokenize(clean_text) if x not in stopwords.words('english')])

  final_text.append(clean_text)



In [149]:
final_text[:5]

['go jurong point crazy available bugis great world la buffet cine got amore wat',
 'ok lar joking wif oni',
 'free entry wkly comp win fa cup final tkts st may text fa receive entry question std txt rate apply',
 'dun say early hor already say',
 'nah think goes usf lives around though']

In [150]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [151]:
# calculating words of bags
WOB_matrix = CountVectorizer(max_features=2500, max_df=0.8).fit_transform(final_text).toarray()
WOB_matrix.shape

(5572, 2500)

In [152]:
xtrain, xtest, ytrain, ytest = train_test_split(WOB_matrix, target, test_size=0.2, random_state=0)

Implementing Descision Tree

In [153]:
dsc_clf = DecisionTreeClassifier().fit(xtrain, ytrain)

In [154]:
ytest[:10], dsc_clf.predict(xtest)[:10]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]))

In [155]:
print(classification_report(ytest, dsc_clf.predict(xtest)))

              precision    recall  f1-score   support

           0       0.97      0.99      0.98       949
           1       0.93      0.84      0.88       166

    accuracy                           0.97      1115
   macro avg       0.95      0.91      0.93      1115
weighted avg       0.97      0.97      0.97      1115



In [156]:
confusion_matrix(ytest, dsc_clf.predict(xtest)),confusion_matrix(ytrain, dsc_clf.predict(xtrain))

(array([[938,  11],
        [ 27, 139]]), array([[3876,    0],
        [   2,  579]]))

In [157]:
accuracy_score(ytest, dsc_clf.predict(xtest)),accuracy_score(ytrain, dsc_clf.predict(xtrain))

(0.9659192825112107, 0.9995512676688355)

Implementing Random Forest Classifier

In [158]:
rf_clf = RandomForestClassifier(n_estimators=150).fit(xtrain, ytrain)

In [159]:
ytest[:10], rf_clf.predict(xtest)[:10]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

In [160]:
print(classification_report(ytest, rf_clf.predict(xtest)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.99       949
           1       1.00      0.85      0.92       166

    accuracy                           0.98      1115
   macro avg       0.99      0.92      0.95      1115
weighted avg       0.98      0.98      0.98      1115



In [161]:
confusion_matrix(ytest, rf_clf.predict(xtest)),confusion_matrix(ytrain, rf_clf.predict(xtrain))

(array([[949,   0],
        [ 25, 141]]), array([[3876,    0],
        [   2,  579]]))

In [162]:
accuracy_score(ytest, rf_clf.predict(xtest)),accuracy_score(ytrain, rf_clf.predict(xtrain))

(0.9775784753363229, 0.9995512676688355)

multinomial Naive Bayes classifier

In [163]:
nb_clf = MultinomialNB().fit(xtrain, ytrain)

In [164]:
ytest[:10], nb_clf.predict(xtest)[:10]

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]), array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

In [165]:
print(classification_report(ytest, nb_clf.predict(xtest)))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       949
           1       0.93      0.94      0.94       166

    accuracy                           0.98      1115
   macro avg       0.96      0.96      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [166]:
accuracy_score(ytest, rf_clf.predict(xtest)),accuracy_score(ytrain, rf_clf.predict(xtrain))

(0.9775784753363229, 0.9995512676688355)

Testing

In [172]:
lr.inverse_transform(rf_clf.predict(WOB_matrix[9].reshape(1, -1)))

array(['spam'], dtype=object)

In [173]:
final_text[9]

'mobile months entitled update latest colour mobiles camera free call mobile update co free'

In [174]:
features[9]

'Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030'

In [178]:
df.iloc[9,:]

result                                                   spam
features    Had your mobile 11 months or more? U R entitle...
unamed1                                                   NaN
unamed2                                                   NaN
unamed3                                                   NaN
Name: 9, dtype: object